In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.chart import PieChart, Reference, BarChart
from openpyxl.chart.series import DataPoint
from openpyxl.drawing.fill import ColorChoice
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.chart import PieChart, Reference, BarChart
from openpyxl.chart.series import DataPoint
from openpyxl.drawing.fill import ColorChoice
import matplotlib.pyplot as plt

def analyze_tv_programs(input_file, output_path):
    """
    Analyze TV programs data and generate required metrics
    """
    try:
        # Read the Excel file
        print("Reading Excel file...")
        df = pd.read_excel(input_file)
        
        # Clean column names (remove any extra spaces)
        df.columns = df.columns.str.strip()
        
        # Display basic info about the dataset
        print(f"Dataset shape: {df.shape}")
        print(f"Columns: {list(df.columns)}")
        
        # Check for missing values in key columns
        print("\nChecking for missing values...")
        key_columns = ['series_name', 'airing_data', 'delivery_date']
        for col in key_columns:
            if col in df.columns:
                missing_count = df[col].isna().sum()
                print(f"{col}: {missing_count} missing values")
        
        # ============ PROGRAMS ANALYSIS ============
        print("\n=== PROGRAMS ANALYSIS ===")
        
        # Total programs
        total_programs = len(df)
        print(f"Total programs: {total_programs}")
        
        # Delivered programs (assuming programs with delivery_date are delivered)
        delivered_programs = df['delivery_date'].notna().sum()
        print(f"Delivered programs: {delivered_programs}")
        
        # Remaining programs
        remaining_programs = total_programs - delivered_programs
        print(f"Remaining programs: {remaining_programs}")
        
        # ============ CHANNELS ANALYSIS ============
        print("\n=== CHANNELS ANALYSIS ===")
        
        # Function to split channels from airing_data
        def extract_channels(airing_data):
            """Extract individual channels from airing_data string"""
            if pd.isna(airing_data):
                return []
            
            # Convert to string and clean
            airing_str = str(airing_data).strip()
            
            # Remove any extra spaces and normalize
            airing_str = ' '.join(airing_str.split())
            
            # If it's empty after cleaning, return empty list
            if not airing_str:
                return []
            
            # Try different delimiters in order of preference
            delimiters = [',', ';', '|', '/', '\\']
            channels = [airing_str]  # Start with the whole string
            
            for delimiter in delimiters:
                if delimiter in airing_str:
                    channels = airing_str.split(delimiter)
                    break
            
            # Clean each channel and filter out empty/invalid ones
            cleaned_channels = []
            for channel in channels:
                channel = channel.strip()
                # Filter out empty strings and very short strings that might be noise
                if channel and len(channel) >= 2:
                    # Remove any trailing/leading special characters
                    channel = channel.strip('.,;|/\\()[]{}')
                    if channel:
                        cleaned_channels.append(channel)
            
            return cleaned_channels
        
        # Create a list to store individual channel records
        channel_records = []
        
        print("Processing channels from airing_data...")
        for idx, row in df.iterrows():
            channels = extract_channels(row['airing_data'])
            
            # If no channels found, skip this row for channel analysis
            if not channels:
                continue
                
            for channel in channels:
                # Additional cleaning for channel names
                channel = channel.strip().upper()  # Normalize to uppercase
                
                # Skip if channel is too short or contains only numbers/special chars
                if len(channel) < 2 or channel.isdigit():
                    continue
                    
                channel_record = {
                    'channel': channel,
                    'series_name': row['series_name'],
                    'delivered': pd.notna(row['delivery_date']),
                    'delivery_date': row['delivery_date'],
                    'original_row': idx
                }
                channel_records.append(channel_record)
        
        # Create DataFrame from channel records
        channels_df = pd.DataFrame(channel_records)
        
        if len(channels_df) > 0:
            print(f"Total channel-program combinations: {len(channels_df)}")
            
            # Group by channel to get channel-level metrics
            channel_summary = channels_df.groupby('channel').agg({
                'delivered': ['count', 'sum'],
                'series_name': 'nunique'
            }).reset_index()
            
            # Flatten column names
            channel_summary.columns = ['channel', 'total_programs', 'delivered_programs', 'unique_series']
            
            # Calculate completion percentage
            channel_summary['completion_percentage'] = (
                channel_summary['delivered_programs'] / channel_summary['total_programs'] * 100
            ).round(2)
            
            # Filter out channels with very few programs (likely noise)
            channel_summary = channel_summary[channel_summary['total_programs'] >= 1]
            
            # Channel metrics
            total_channels = len(channel_summary)
            completed_channels = (channel_summary['completion_percentage'] == 100.0).sum()
            pending_channels = total_channels - completed_channels
            
            print(f"Total unique channels: {total_channels}")
            print(f"Completed channels (100%): {completed_channels}")
            print(f"Pending channels: {pending_channels}")
            
            # Focus analysis
            near_completion = ((channel_summary['completion_percentage'] >= 90.0) & 
                             (channel_summary['completion_percentage'] < 100.0)).sum()
            focus_needed = (channel_summary['completion_percentage'] < 90.0).sum()
            
            print(f"Near completion (≥90%): {near_completion}")
            print(f"Focus needed (<90%): {focus_needed}")
            
            # Top 20 channels to focus on
            focus_channels = channel_summary[
                channel_summary['completion_percentage'] < 100.0
            ].nsmallest(20, 'completion_percentage')
            
            print(f"Next top channels to focus: {min(20, len(focus_channels))}")
            
        else:
            print("No valid channels found in airing_data")
            total_channels = 0
            completed_channels = 0
            pending_channels = 0
            near_completion = 0
            focus_needed = 0
        
        # ============ CREATE OUTPUT ============
        print("\n=== CREATING OUTPUT ===")
        
        # Create results DataFrame
        results_data = [
            ['Programs', 'Total', total_programs],
            ['Programs', 'Delivered', delivered_programs],
            ['Programs', 'Remaining', remaining_programs],
            ['Channels', 'Total', total_channels],
            ['Channels', 'Completed', completed_channels],
            ['Channels', 'Pending', pending_channels],
            ['Focus', 'Near-Completion (≥90 %)', near_completion],
            ['Focus', 'Focus (<90 %)', focus_needed],
            ['Focus', 'Next Top Channels to focus', min(20, len(focus_channels) if 'focus_channels' in locals() else 0)]
        ]
        
        results_df = pd.DataFrame(results_data, columns=['Section', 'Metric', 'Value'])
        
        # Create output directory if it doesn't exist
        os.makedirs(output_path, exist_ok=True)
        
        # Save main results
        output_file = os.path.join(output_path, 'analysis_results.xlsx')
        
        # First save with pandas
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            # Main results
            results_df.to_excel(writer, sheet_name='Summary', index=False)
            
            # Detailed channel analysis (if available)
            if len(channels_df) > 0:
                channel_summary_sorted = channel_summary.sort_values('completion_percentage')
                channel_summary_sorted.to_excel(writer, sheet_name='Channel_Details', index=False)
                
                # Top channels to focus
                if len(focus_channels) > 0:
                    focus_channels_output = focus_channels[['channel', 'total_programs', 'delivered_programs', 'completion_percentage']].copy()
                    focus_channels_output = focus_channels_output.sort_values('completion_percentage')
                    focus_channels_output.to_excel(writer, sheet_name='Focus_Channels', index=False)
            
            # Original data sample
            df.head(100).to_excel(writer, sheet_name='Original_Data_Sample', index=False)
        
        # Now add charts using openpyxl
        print("Adding charts to Excel file...")
        try:
            # Load the workbook
            wb = load_workbook(output_file)
            ws = wb['Summary']
            
            # Create chart data for donut chart
            chart_data = [
                ['Status', 'Count'],
                ['Remaining', remaining_programs],
                ['Delivered', delivered_programs]
            ]
            
            # Add chart data to the worksheet starting from column E
            for row_idx, row_data in enumerate(chart_data, start=1):
                for col_idx, value in enumerate(row_data, start=5):  # Column E = 5
                    ws.cell(row=row_idx, column=col_idx, value=value)
            
            # Create donut chart (which is a pie chart with a hole)
            donut_chart = PieChart()
            data = Reference(ws, min_col=6, min_row=2, max_row=3)  # Column F, rows 2-3
            categories = Reference(ws, min_col=5, min_row=2, max_row=3)  # Column E, rows 2-3
            
            donut_chart.add_data(data)
            donut_chart.set_categories(categories)
            donut_chart.title = f"{delivered_programs/total_programs*100:.1f}% Delivered"
            donut_chart.height = 10
            donut_chart.width = 15
            
            # Make it a donut chart by setting the hole size
            donut_chart.holeSize = 50  # Percentage of the hole size (10-90)
            
            # Customize colors using hex values directly
            try:
                # Blue for remaining (5B9BD5)
                # Red for delivered (C65853)
                colors = ['5B9BD5', 'C65853']
                for i, color in enumerate(colors):
                    props = DataPoint(idx=i)
                    props.graphicalProperties.solidFill = color
                    donut_chart.series[0].data_points = [props]
            except Exception as e:
                print(f"Note: Could not set custom colors: {e}")
            
            # Add chart to worksheet
            ws.add_chart(donut_chart, "H2")
            
            # Add a second chart for completed channels if we have channel data
            if len(channels_df) > 0:
                # Get top 10 completed channels for horizontal bar chart
                top_completed = channel_summary[channel_summary['completion_percentage'] == 100.0].nlargest(10, 'total_programs')
                
                if len(top_completed) > 0:
                    # Add chart title and data starting from row 20
                    ws.cell(row=20, column=5, value="Completed Channels (Total Records)")
                    
                    # Add data for bar chart
                    for idx, (_, row) in enumerate(top_completed.iterrows(), start=21):
                        ws.cell(row=idx, column=5, value=row['channel'])
                        ws.cell(row=idx, column=6, value=row['total_programs'])
                    
                    # Create horizontal bar chart
                    bar_chart = BarChart()
                    bar_chart.type = "bar"
                    bar_chart.style = 10
                    bar_chart.title = "Completed Channels (Total Records)"
                    bar_chart.y_axis.title = 'Channels'
                    bar_chart.x_axis.title = 'Total Programs'
                    
                    data = Reference(ws, min_col=6, min_row=21, max_row=20+len(top_completed))
                    categories = Reference(ws, min_col=5, min_row=21, max_row=20+len(top_completed))
                    
                    bar_chart.add_data(data)
                    bar_chart.set_categories(categories)
                    bar_chart.height = 12
                    bar_chart.width = 18
                    ws.add_chart(bar_chart, "H25")
            
            # Save the workbook with charts
            wb.save(output_file)
            print("Charts added successfully!")
            
        except Exception as e:
            print(f"Error adding charts: {e}")
            import traceback
            traceback.print_exc()
        
        print(f"\nResults saved to: {output_file}")
        
        # Display results
        print("\n=== FINAL RESULTS ===")
        print(results_df.to_string(index=False))
        
        # Additional insights
        if len(channels_df) > 0:
            print(f"\n=== ADDITIONAL INSIGHTS ===")
            print(f"Average completion percentage: {channel_summary['completion_percentage'].mean():.2f}%")
            print(f"Median completion percentage: {channel_summary['completion_percentage'].median():.2f}%")
            
            # Show sample of channels and their completion rates
            print("\nSample of channel completion rates:")
            sample_channels = channel_summary.head(10)[['channel', 'total_programs', 'delivered_programs', 'completion_percentage']]
            print(sample_channels.to_string(index=False))
        
        return results_df
        
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# Main execution
if __name__ == "__main__":
    # File paths
    input_file = r"C:\Users\Aaryan\Documents\combined\combined.xlsx"
    output_path = r"C:\Users\Aaryan\Documents\combined\output"
    
    # Check if input file exists
    if not os.path.exists(input_file):
        print(f"Error: Input file not found at {input_file}")
        print("Please check the file path and try again.")
    else:
        print(f"Processing file: {input_file}")
        print(f"Output will be saved to: {output_path}")
        
        # Run analysis
        results = analyze_tv_programs(input_file, output_path)
        
        if results is not None:
            print("\n✅ Analysis completed successfully!")
            print(f"📊 Results saved to: {os.path.join(output_path, 'analysis_results.xlsx')}")
        else:
            print("\n❌ Analysis failed!")